### wworking to extract the tables on the URL of the year

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [78]:
url = "http://www.planecrashinfo.com/database.htm"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

links = []
#assign the links scraped into the var links
for link in soup.find_all("a"):
    if not link.get("href").startswith("/"):
        links.append("http://www.planecrashinfo.com/" + link.get("href"))
    else:
        links.append("http://www.planecrashinfo.com" + link.get("href"))

#remoeve the last 4 strings from each url
links_sliced = [link[:-4] for link in links]

#append -1 to -150 to each url, do to the way the urls are constructed can't find a better solution
new_links = []
for link in links_sliced:
    for i in range(-1, -151, -1):
        new_link = link + str(i) + str('.htm')
        new_links.append(new_link)

In [ ]:
combined_df = pd.DataFrame() # empty dataframe

for url in new_links:
    response = requests.get(url)
    time.sleep(random.uniform(0, 2)) #random delay
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    if tables:
        tablll = pd.read_html(str(tables[0]), flavor='bs4')[0] #look for the table and assign to the tablll var
        combined_df = pd.concat([combined_df, tablll], ignore_index=True) #merge all the tables from each url into the combined_df dataframe
    else:
        print(f"No tables found on {url}") #print an error if no table is found

In [83]:
combined_df.to_excel('data.xlsx', index=False)

### below is code, need to change cell for code 

url = "http://www.planecrashinfo.com/1920/1920.htm"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
base_url = "http://www.planecrashinfo.com/1920/"

# Empty list to store the URLs
urls = []

for link in soup.find_all("a"):
    link_url = link.get("href")
    if link_url:
        full_url = base_url + link_url
        urls.append(full_url)

# empty dataframe
combined_df = pd.DataFrame()

for url in urls:
    response = requests.get(url)  # change urls to url
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    if tables:
        tablll = pd.read_html(str(tables[0]), flavor='bs4')[0]
        combined_df = pd.concat([combined_df, tablll], ignore_index=True)
    else:
        print(f"No tables found on {url}")

In [36]:
combined_df

,0,1
0,-,ACCIDENT DETAILS
1,Date:,"September 17, 1908"
2,Time:,1718
3,Location:,"Fort Myer, Virginia"
4,Operator:,Military - U.S. Army
...,...,...
709,cn / ln:,57
710,Aboard:,1 (passengers:0 crew:1)
711,Fatalities:,1 (passengers:0 crew:1)
712,Ground:,0


header_row = 0
combined_df.columns = combined_df.iloc[header_row]

# Convert row to column header using DataFrame.iloc[]
combined_df.columns = combined_df.iloc[0]
combined_df